In [6]:
import os
import torch

base_name = 'random16M_vocab32_1K'
model = base_name + ".pth"
ckpt_path = os.path.join("eval_models",model)
checkpoint = torch.load(ckpt_path,map_location = "cpu")
print(checkpoint['iter_num'])
print(checkpoint.keys())
print(checkpoint['model_args'])


/tmp/ipykernel_71183/3730475234.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path,map_location = "cpu")


FileNotFoundError: [Errno 2] No such file or directory: 'eval_models/random16M_vocab32_1K.pth'

In [14]:
import os
import json

def clean_evaluation_for_model(model_name, 
                               elo_results_file='elo_results.json',
                               quick_save_file='quick_save.json'):
    """
    Removes all entries from 'elo_results.json' and 'quick_save.json'
    pertaining to the specified model_name, so you can start a fresh evaluation.
    """

    def load_json_if_exists(filepath):
        if os.path.exists(filepath):
            print("file exists")
            with open(filepath, 'r') as f:
                return json.load(f)
        return {}

    def save_json(filepath, data):
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4)

    # --- Clean from elo_results_file ---
    elo_results = load_json_if_exists(elo_results_file)
    # We loop over keys in a list(...) to avoid changing the dict while iterating
    for key in list(elo_results.keys()):
        # If this entry was stored for the given model_name, remove it
        if "model" in elo_results[key]:
            if elo_results[key]["model"] == (model_name):
                print("found the entry and deleting it")
                del elo_results[key]
    
    # Save back to file
    save_json(elo_results_file, elo_results)

    # --- Clean from quick_save_file ---
    quick_save = load_json_if_exists(quick_save_file)
    for key in list(quick_save.keys()):
        # If this entry pertains to the given model_name, remove it
        if "games_so_far" in quick_save[key] and "current_elo" in quick_save[key]:
            # If you store the actual model name inside each entry, you can match it similarly
            # or if the key pattern includes the model_name, you can do:
            # if key.startswith(model_name):
            #     del quick_save[key]
            #
            # But if you keep a "model" field:
            # if quick_save[key]["model"] == model_name:
            #     del quick_save[key]
            #
            # For brevity, let's just match if the key starts with model_name:
            if key.startswith(model_name):
                del quick_save[key]

    # Save back to file
    save_json(quick_save_file, quick_save)

    print(f"All data for model '{model_name}' has been removed from '{elo_results_file}' and '{quick_save_file}'.")

clean_evaluation_for_model("karvmodel_600K.pth")

file exists
found the entry and deleting it
file exists
All data for model 'karvmodel_600K.pth' has been removed from 'elo_results.json' and 'quick_save.json'.
